# Import

In [15]:
%pip install spotipy
%pip install pandas
#git clone https://github.com/JayChen35/spotify-to-mp3-python.git
%pip install yt_dlp
!pip3 install spotipy && pip3 install youtube_dl && pip3 install youtube_search && pip3 install yt_dlp && pip3 install ffprobe && pip3 install ffmpeg

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [16]:
import json
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import re


import time

# Load data

## Data

In [17]:
df = pd.read_csv('Data/Data/data_incial_completa.csv').astype({ 'valence_tags': np.float32, 'arousal_tags':np.float32, 'dominance_tags':np.float32})

In [18]:
df

,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,cuadrante
0,https://www.last.fm/music/eminem/_/%2527till%2...,'Till I Collapse,Eminem,aggressive,6,4.550000,5.273125,5.690625,cab93def-26c5-4fb0-bedd-26ec4c1619e1,4xkOaSrkexMciUUogZKVTS,rap,3
1,https://www.last.fm/music/metallica/_/st.%2banger,St. Anger,Metallica,aggressive,8,3.710000,5.833000,5.427250,727a2529-7ee8-4860-aef6-7959884895cb,3fOc9x06lKJBhz435mInlH,metal,3
2,https://www.last.fm/music/rick%2bross/_/speedi...,Speedin',Rick Ross,aggressive,1,3.080000,5.870000,5.490000,NaN,3Y96xd4Ce0J47dcalLrEC8,rap,3
3,https://www.last.fm/music/m.i.a./_/bamboo%2bbanga,Bamboo Banga,M.I.A.,aggressive,13,6.555071,5.537214,5.691357,99dd2c8c-e7c1-413e-8ea4-4497a00ffa18,6tqFC1DIOphJkCwrjVzPmg,hip-hop,3
4,https://www.last.fm/music/dope/_/die%2bmf%2bdie,Die MF Die,Dope,aggressive,7,3.771177,5.348235,5.441765,b9eb3484-5e0e-4690-ab5a-ca91937032a5,5bU4KX47KqtDKKaLM4QCzh,metal,3
...,...,...,...,...,...,...,...,...,...,...,...,...
61625,https://www.last.fm/music/quietdrive/_/secret,Secret,Quietdrive,transparent,1,5.370000,3.450000,5.330000,909ba2e0-6076-4ad8-8be9-a335725de4ef,2bRIsZ92JRKlvQOZlyR9CO,NaN,0
61626,https://www.last.fm/music/medications/_/the%2b...,The Last of the Rest Was the End,Medications,transparent,1,5.370000,3.450000,5.330000,ffd10ae8-858a-4b5e-819e-64f1174bbd42,7o3Np7cho9cBCrNDokxzYC,NaN,0
61627,https://www.last.fm/music/daniel%2blanois/_/lo...,Lovechild,Daniel Lanois,transparent,2,6.685000,4.405000,5.625000,16c3d394-c4d4-4dc2-bbf1-b2bef3ac861c,4fVObxldDzxxRD6a5Eth9s,indie,0
61628,https://www.last.fm/music/tapage/_/last%2binhale,Last Inhale,Tapage,transparent,6,4.301667,3.341667,4.466667,b744fdbc-7b4c-49a1-a3d6-99047d82eb34,5WxwRwUQ4R4L46VEm3213y,ambient,0


# Audio features

In [19]:
#Authentication - without user
cid = 'ade5fdc8052444e1b7d3bf27aa5f348a'#'ade5fdc8052444e1b7d3bf27aa5f348a'#'dfa416f0aec64188a6ef0895bedc85a0'
secret = 'ae35edfc4f614ecdb69c3508c40a88de'#'ae35edfc4f614ecdb69c3508c40a88de'#'161e40a7005642b7896fee1998ace95a'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

## Low features

In [20]:
k = 58944
tam = df["track"].size

track_uri  = []
track_name = []
artist_uri = []
artist_info = []
artist_name = []
artist_pop = []
artist_genres = []
album  = []
track_pop = []

danceability = []
energy = []
key =  []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
id_song = []
uri = []
track_href = []
analysis_url = []
duration_ms = []
time_signature = []
dataCancion = []

In [21]:
#for k, song in enumerate(df['spotify_id']):
while (k < tam):
    print("k: " + str(k) + " id: "+ str(df['spotify_id'][k]))
    try:
        cancion = (sp.track('spotify:track:' + str(df['spotify_id'][k])))
        
        if(cancion != None):
            file_name = "Data/Features/Song/" + str(k+1) + "-" + str(df['spotify_id'][k]).replace(" ","-").replace("/","-") + "-" + str(df["artist"].iloc[k]).replace(" ","-").replace("/","-") + ".json"
            with open(file_name, "w") as f:
                json.dump(cancion, f)


            #URI
            track_uri= np.concatenate([track_uri, [cancion["uri"]]])
            
            #Track name
            track_name= np.concatenate([track_name, [cancion["name"]]])

            
            #Main Artist
            artist_uri= np.concatenate([artist_uri, [cancion["artists"][0]["uri"]]])
            infoArtist = sp.artist(cancion["artists"][0]["uri"])
            artist_info= np.concatenate([artist_info, [infoArtist]])
            
            #Name, popularity, genre
            artist_name= np.concatenate([artist_name, [cancion["artists"][0]["name"]]])
            artist_pop= np.concatenate([artist_pop, [infoArtist["popularity"]]])
            #artist_genres= np.concatenate([artist_genres, infoArtist["genres"]])
            
            #Album
            album= np.concatenate([album, [cancion["album"]["name"]]])
            
            #Popularity of the track
            track_pop= np.concatenate([track_pop, [cancion["popularity"]]])

            dataCancion_ = sp.audio_features(cancion["uri"])[0]

            if(dataCancion_ != None):
                dataCancion = np.concatenate([dataCancion, [dataCancion_]])
                
                file_name = "Data/Features/Low/" + str(k+1) + "-" + str(df['spotify_id'][k]).replace(" ","-").replace("/","-") + "-" + str(df["artist"].iloc[k]).replace(" ","-").replace("/","-") + ".json"
                with open(file_name, "w") as f:
                    json.dump(dataCancion_, f)

                danceability= np.concatenate([danceability, [dataCancion_['danceability']]])
                energy= np.concatenate([energy, [dataCancion_['energy']]])
                key= np.concatenate([key, [dataCancion_['key']]])
                loudness= np.concatenate([loudness, [dataCancion_['loudness']]])
                mode= np.concatenate([mode, [dataCancion_['mode']]])
                speechiness= np.concatenate([speechiness, [dataCancion_['speechiness']]])
                acousticness= np.concatenate([acousticness, [dataCancion_['acousticness']]])
                instrumentalness= np.concatenate([instrumentalness, [dataCancion_['instrumentalness']]])
                liveness= np.concatenate([liveness, [dataCancion_['liveness']]])
                valence= np.concatenate([valence, [dataCancion_['valence']]])
                tempo= np.concatenate([tempo, [dataCancion_['tempo']]])
                id_song= np.concatenate([id_song, [dataCancion_['id']]])
                uri= np.concatenate([uri, [dataCancion_['uri']]])
                track_href= np.concatenate([track_href, [dataCancion_['track_href']]])
                analysis_url= np.concatenate([analysis_url, [dataCancion_['analysis_url']]])
                duration_ms= np.concatenate([duration_ms, [dataCancion_['duration_ms']]])
                time_signature= np.concatenate([time_signature, [dataCancion_['time_signature']]])
            else:
                dataCancion = np.concatenate(np.nan)
                
                file_name = "Data/Features/Low/" + str(k+1) + "-NONE-" + str(df['spotify_id'][k]).replace(" ","-").replace("/","-") + "-" + str(df["artist"].iloc[k]).replace(" ","-").replace("/","-") + ".json"
                with open(file_name, "w") as f:
                    json.dump('NONE', f)

                danceability= np.concatenate(np.nan)
                energy= np.concatenate(np.nan)
                key= np.concatenate(np.nan)
                loudness= np.concatenate(np.nan)
                mode= np.concatenate(np.nan)
                speechiness= np.concatenate(np.nan)
                acousticness= np.concatenate(np.nan)
                instrumentalness= np.concatenate(np.nan)
                liveness= np.concatenate(np.nan)
                valence= np.concatenate(np.nan)
                tempo= np.concatenate(np.nan)
                id_song= np.concatenate(np.nan)
                uri= np.concatenate(np.nan)
                track_href= np.concatenate(np.nan)
                analysis_url= np.concatenate(np.nan)
                duration_ms= np.concatenate(np.nan)
                time_signature= np.concatenate(np.nan)
        else:
            file_name = "Data/Features/Song/" + str(k+1) + "-NONE-" + str(df['spotify_id'][k]).replace(" ","-").replace("/","-") + "-" + str(df["artist"].iloc[k]).replace(" ","-").replace("/","-") + ".json"
            with open(file_name, "w") as f:
                json.dump('NONE', f)


            #URI
            track_uri= np.concatenate(np.nan)
            
            #Track name
            track_name= np.concatenate(np.nan)

            
            #Main Artist
            artist_uri= np.concatenate(np.nan)
            infoArtist = sp.artist(np.nan)
            artist_info= np.concatenate(np.nan)
            
            #Name, popularity, genre
            artist_name= np.concatenate(np.nan)
            artist_pop= np.concatenate(np.nan)            
            
            #Album
            album= np.concatenate(np.nan)
            
            #Popularity of the track
            track_pop= np.concatenate(np.nan)

            

            dataCancion = np.concatenate(np.nan)
                
            file_name = "Data/Features/Low/" + str(k+1) + "-NONE-" + str(df['spotify_id'][k]).replace(" ","-").replace("/","-") + "-" + str(df["artist"].iloc[k]).replace(" ","-").replace("/","-") + ".json"
            with open(file_name, "w") as f:
                json.dump('NONE', f)

            danceability= np.concatenate(np.nan)
            energy= np.concatenate(np.nan)
            key= np.concatenate(np.nan)
            loudness= np.concatenate(np.nan)
            mode= np.concatenate(np.nan)
            speechiness= np.concatenate(np.nan)
            acousticness= np.concatenate(np.nan)
            instrumentalness= np.concatenate(np.nan)
            liveness= np.concatenate(np.nan)
            valence= np.concatenate(np.nan)
            tempo= np.concatenate(np.nan)
            id_song= np.concatenate(np.nan)
            uri= np.concatenate(np.nan)
            track_href= np.concatenate(np.nan)
            analysis_url= np.concatenate(np.nan)
            duration_ms= np.concatenate(np.nan)
            time_signature= np.concatenate(np.nan)

            
        audio_features_low = {
            "track_uri": track_uri,
            "track_name": track_name,
            "artist_uri": artist_uri,
            "artist_info": artist_info,
            "artist_name": artist_name,
            "artist_pop": artist_pop,
            "album" : album,
            "track_pop": track_pop,
            "danceability": danceability,
            "energy": energy,
            "key": key,
            "loudness": loudness,
            "mode": mode,
            "speechiness": speechiness,
            "acousticness": acousticness,
            "instrumentalness": instrumentalness,
            "liveness": liveness,
            "valence": valence,
            "tempo": tempo,
            "id_song": id_song,
            "uri": uri,
            "track_href": track_href,
            "analysis_url": analysis_url,
            "duration_ms": duration_ms,
            "time_signature": time_signature,
            "dataCancion": dataCancion

        }
        audio_features_low = pd.DataFrame(audio_features_low)

        audio_features_low.to_csv('Data/Data/data_caracteristicas_audio_low_26.csv', header=True, index=False)

        k = k + 1
        

    except:
        time.sleep(100)


k: 58944 id: 4VbfHL1I4DzlY6VlsFgbpy
k: 58945 id: 1fT6Tf7iX1gcKMs31qkrOB
k: 58946 id: 4q8f3Ls32PcXE8Ks4V1ufx
k: 58947 id: 5rRXNVSsNZi4rXgVzdNGiE
k: 58948 id: 0517GHmfOrssOv3Pe31cdS
k: 58949 id: 09ur0bERAaelsMsUqhGsG4
k: 58950 id: 4eoSLFmHn2rGa8uMoizCT2
k: 58951 id: 0srJOgR9iEya96bqWPpzlb
k: 58952 id: 54UKJ1LNkvl6R3uHyPrrYn
k: 58953 id: 5QpppkxpoISpjvEuI7bVmc
k: 58954 id: 1YcbcXrUh991g49u1Wn0g0
k: 58955 id: 66DoJfUjtI3spgjqBCBkic
k: 58956 id: 2YLjhEFeIiokzK1yYprih4
k: 58957 id: 04mtaY2jCIFtzin1iYsVdh
k: 58958 id: 4ze9kD9Ui7qH23HKn5Uevl
k: 58959 id: 6goxXvbkEo0EKM5O6Cp0tc
k: 58960 id: 5A5xAcQywDqzT2fCDFUU2L
k: 58961 id: 1sTfYqilqlXrU08PHsLO8e
k: 58962 id: 1rc9CwsjtdDclx3aJKWTNz
k: 58963 id: 75jalTVnrHPg816taJNve8
k: 58964 id: 7DywNNMgrbBbhGvQkNRrAf
k: 58965 id: 0uTUXooR8avbvsmaP8m2RA
k: 58966 id: 5CNVLKxoVv3V5rCkAqjZlS
k: 58967 id: 1BLM0GcEAwjDvZDRt0Rtdr
k: 58968 id: 2rqsNfSeM7Rkv6AbzDG499
k: 58969 id: 2eBUgXa4ff60j80DTcFL9T
k: 58970 id: 3eP8PPhLtH5su57Rx6icJx
k: 58971 id: 0NqgCp0PlMg8Mgk

In [22]:
audio_features_low

,track_uri,track_name,artist_uri,artist_info,artist_name,artist_pop,album,track_pop,danceability,energy,...,liveness,valence,tempo,id_song,uri,track_href,analysis_url,duration_ms,time_signature,dataCancion
0,spotify:track:4VbfHL1I4DzlY6VlsFgbpy,Real Sharp Twig,spotify:artist:7d0wPHbHFUXnwnP3BQVqaV,{'external_urls': {'spotify': 'https://open.sp...,Keshco,0.0,Double Deadly,0.0,0.418,0.38600,...,0.2100,0.2110,108.132,4VbfHL1I4DzlY6VlsFgbpy,spotify:track:4VbfHL1I4DzlY6VlsFgbpy,https://api.spotify.com/v1/tracks/4VbfHL1I4Dzl...,https://api.spotify.com/v1/audio-analysis/4Vbf...,257290.0,4.0,"{'danceability': 0.418, 'energy': 0.386, 'key'..."
1,spotify:track:1fT6Tf7iX1gcKMs31qkrOB,"Wozzeck, Op. 7: Act II Scene 2: Street in town",spotify:artist:60ju8DuNEmkdLw3ymddLje,{'external_urls': {'spotify': 'https://open.sp...,Alban Berg,29.0,"Berg, A.: Wozzeck",0.0,0.370,0.09170,...,0.1040,0.1980,81.717,1fT6Tf7iX1gcKMs31qkrOB,spotify:track:1fT6Tf7iX1gcKMs31qkrOB,https://api.spotify.com/v1/tracks/1fT6Tf7iX1gc...,https://api.spotify.com/v1/audio-analysis/1fT6...,636120.0,4.0,"{'danceability': 0.37, 'energy': 0.0917, 'key'..."
2,spotify:track:4q8f3Ls32PcXE8Ks4V1ufx,"Chopin: 24 Preludes, Op. 28: No. 15 in D-Flat ...",spotify:artist:7y97mc3bZRFXzT2szRM4L4,{'external_urls': {'spotify': 'https://open.sp...,Frédéric Chopin,74.0,"Chopin : Preludes, Ballades Nos 3 & 4, Nocturnes",52.0,0.398,0.00293,...,0.0897,0.0804,130.464,4q8f3Ls32PcXE8Ks4V1ufx,spotify:track:4q8f3Ls32PcXE8Ks4V1ufx,https://api.spotify.com/v1/tracks/4q8f3Ls32PcX...,https://api.spotify.com/v1/audio-analysis/4q8f...,348333.0,4.0,"{'danceability': 0.398, 'energy': 0.00293, 'ke..."
3,spotify:track:5rRXNVSsNZi4rXgVzdNGiE,Let Me Tell You Something,spotify:artist:7xmWDToWl9RN1pqSsYSRFf,{'external_urls': {'spotify': 'https://open.sp...,Sophia Ramos,1.0,Revelator,0.0,0.623,0.81000,...,0.6930,0.7170,125.375,5rRXNVSsNZi4rXgVzdNGiE,spotify:track:5rRXNVSsNZi4rXgVzdNGiE,https://api.spotify.com/v1/tracks/5rRXNVSsNZi4...,https://api.spotify.com/v1/audio-analysis/5rRX...,167093.0,4.0,"{'danceability': 0.623, 'energy': 0.81, 'key':..."
4,spotify:track:0517GHmfOrssOv3Pe31cdS,Reject (No Reaction Time),spotify:artist:0WwGwpgafQeLpsp6CL5Ws5,{'external_urls': {'spotify': 'https://open.sp...,Carlo Barbagallo,21.0,Quarter Century,0.0,0.457,0.27000,...,0.2000,0.0525,120.088,0517GHmfOrssOv3Pe31cdS,spotify:track:0517GHmfOrssOv3Pe31cdS,https://api.spotify.com/v1/tracks/0517GHmfOrss...,https://api.spotify.com/v1/audio-analysis/0517...,297000.0,4.0,"{'danceability': 0.457, 'energy': 0.27, 'key':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,spotify:track:2bRIsZ92JRKlvQOZlyR9CO,Secret,spotify:artist:5L1n6wVJMxBw6sZbxD0XAU,{'external_urls': {'spotify': 'https://open.sp...,Quietdrive,40.0,Deliverance,7.0,0.396,0.91500,...,0.1350,0.0734,167.996,2bRIsZ92JRKlvQOZlyR9CO,spotify:track:2bRIsZ92JRKlvQOZlyR9CO,https://api.spotify.com/v1/tracks/2bRIsZ92JRKl...,https://api.spotify.com/v1/audio-analysis/2bRI...,258373.0,4.0,"{'danceability': 0.396, 'energy': 0.915, 'key'..."
2682,spotify:track:7o3Np7cho9cBCrNDokxzYC,The Last of the Rest Was the End,spotify:artist:6ZgHswsP4qcUXXtJepAIU2,{'external_urls': {'spotify': 'https://open.sp...,Medications,10.0,Your Favorite People All In One Place,0.0,0.244,0.86600,...,0.0782,0.3400,144.844,7o3Np7cho9cBCrNDokxzYC,spotify:track:7o3Np7cho9cBCrNDokxzYC,https://api.spotify.com/v1/tracks/7o3Np7cho9cB...,https://api.spotify.com/v1/audio-analysis/7o3N...,324000.0,3.0,"{'danceability': 0.244, 'energy': 0.866, 'key'..."
2683,spotify:track:4fVObxldDzxxRD6a5Eth9s,Lovechild,spotify:artist:5S0AJvE9NB1kGrXRfYh690,{'external_urls': {'spotify': 'https://open.sp...,Daniel Lanois,49.0,Here Is What Is,4.0,0.236,0.10700,...,0.1120,0.0720,79.476,4fVObxldDzxxRD6a5Eth9s,spotify:track:4fVObxldDzxxRD6a5Eth9s,https://api.spotify.com/v1/tracks/4fVObxldDzxx...,https://api.spotify.com/v1/audio-analysis/4fVO...,516280.0,4.0,"{'danceability': 0.236, 'energy': 0.107, 'key'

## HIGH features

### Initial

In [23]:
k = 0
tam = df["track"].size

analysis_time = []
input_process = []
num_samples = []
duration = []
sample_md5 = []
offset_seconds = []
window_seconds = []
analysis_sample_rate = []
analysis_channels = []
end_of_fade_in = []
start_of_fade_out = []
loudness = []
tempo = []
tempo_confidence = []
time_signature = []
time_signature_confidence = []
key = []
key_confidence = []
mode = []
mode_confidence = []
codestring = []
code_version = []
echoprintstring = []
echoprint_version = []
synchstring = []
synch_version = []
rhythmstring = []
rhythm_version = []
data_analysis = []

In [24]:
# bars
songId_bars = []
numberBar_bars = []
start_bars = []
duration_bars = []
confidence_bars = []
track_uris_bars = []

In [25]:
songId_beats = []
numberBeat_beats = []
start_beats = []
duration_beats = []
confidence_beats = []
track_uris_beats = []

In [26]:
songId_tatums = []
numberTatum_tatums = []
start_tatums = []
duration_tatums = []
confidence_tatums = []
track_uris_tatums = []

In [27]:
songId_sections = []
numberSection_sections = []
start_sections = []
duration_sections = []
confidence_sections = []

loudness_sections = []
tempo_sections = []
tempo_confidence_sections = []
key_sections = []
key_confidence_sections = []
mode_sections = []
mode_confidence_sections = []
time_signature_sections = []
time_signature_confidence_sections = []

track_uris_sections = []

In [28]:

songId_segments = []
numberSegments_segments = []
start_segments = []
duration_segments = []
confidence_segments = []
loudness_start_segments = []
loudness_max_segments = []
loudness_max_time_segments = []
loudness_end_segments = []
pitches_segments = []
timbre_segments = []
track_uris_segments = []

In [29]:
while (k < tam):

    if  k < tam :
        try:
            data = (sp.audio_analysis(df["spotify_id"].iloc[k]))

            data_analysis = np.concatenate([data_analysis, [data]])
            analysis_time = np.concatenate([analysis_time, [data["meta"]["analysis_time"]]])      
            input_process = np.concatenate([input_process, [data["meta"]["input_process"]]])      
            num_samples = np.concatenate([num_samples, [data["track"]["num_samples"]]])            
            duration = np.concatenate([duration, [data["track"]["duration"]]])     
            sample_md5 = np.concatenate([sample_md5, [data["track"]["sample_md5"]]])      
            offset_seconds = np.concatenate([offset_seconds, [data["track"]["offset_seconds"]]])      
            window_seconds = np.concatenate([window_seconds, [data["track"]["window_seconds"]]])      
            analysis_sample_rate = np.concatenate([analysis_sample_rate, [data["track"]["analysis_sample_rate"]]])      
            analysis_channels = np.concatenate([analysis_channels, [data["track"]["analysis_channels"]]])     
            end_of_fade_in = np.concatenate([end_of_fade_in, [data["track"]["end_of_fade_in"]]])      
            start_of_fade_out = np.concatenate([start_of_fade_out, [data["track"]["start_of_fade_out"]]])      
            loudness = np.concatenate([loudness, [data["track"]["loudness"]]])      
            tempo = np.concatenate([tempo, [data["track"]["tempo"]]])      
            tempo_confidence = np.concatenate([tempo_confidence, [data["track"]["tempo_confidence"]]])      
            time_signature = np.concatenate([time_signature, [data["track"]["time_signature"]]])      
            time_signature_confidence = np.concatenate([time_signature_confidence, [data["track"]["time_signature_confidence"]]])      
            key = np.concatenate([key, [data["track"]["key"]]])      
            key_confidence = np.concatenate([key_confidence, [data["track"]["key_confidence"]]])      
            mode = np.concatenate([mode, [data["track"]["mode"]]])      
            mode_confidence = np.concatenate([mode_confidence, [data["track"]["mode_confidence"]]])      
            codestring = np.concatenate([codestring, [data["track"]["codestring"]]])      
            code_version = np.concatenate([code_version, [data["track"]["code_version"]]])      
            echoprintstring = np.concatenate([echoprintstring, [data["track"]["echoprintstring"]]])      
            echoprint_version = np.concatenate([echoprint_version, [data["track"]["echoprint_version"]]])      
            synchstring = np.concatenate([synchstring, [data["track"]["synchstring"]]])      
            synch_version = np.concatenate([synch_version, [data["track"]["synch_version"]]])      
            rhythmstring = np.concatenate([rhythmstring, [data["track"]["rhythmstring"]]])      
            rhythm_version = np.concatenate([rhythm_version, [data["track"]["rhythm_version"]]])    
            
            for x, bar in enumerate(data["bars"]): 
                
                songId_bars = np.concatenate([songId_bars, [k+1]])   
                numberBar_bars = np.concatenate([numberBar_bars, [x]]) 
                start_bars = np.concatenate([start_bars, [bar["start"]]]) 
                duration_bars = np.concatenate([duration_bars, [bar["duration"]]])   
                confidence_bars = np.concatenate([confidence_bars, [bar["confidence"]]])     
                track_uris_bars = np.concatenate([track_uris_bars, [df["spotify_id"].iloc[k]]])  
            
            for x, beat in enumerate(data["beats"]): 

                songId_beats = np.concatenate([songId_beats, [k+1]])   
                numberBeat_beats = np.concatenate([numberBeat_beats, [x]])   
                start_beats =  np.concatenate([start_beats, [beat["start"]]]) 
                duration_beats = np.concatenate([duration_beats, [beat["duration"]]])   
                confidence_beats = np.concatenate([confidence_beats, [beat["confidence"]]])    
                track_uris_beats = np.concatenate([track_uris_beats, [df["spotify_id"].iloc[k]]])   

            for x, tatum in enumerate(data["tatums"]): 

                songId_tatums = np.concatenate([songId_tatums, [k+1]])  
                numberTatum_tatums = np.concatenate([numberTatum_tatums, [x]])
                start_tatums = np.concatenate([start_tatums, [tatum["start"]]]) 
                duration_tatums = np.concatenate([duration_tatums, [tatum["duration"]]]) 
                confidence_tatums = np.concatenate([confidence_tatums, [tatum["confidence"]]])
                track_uris_tatums = np.concatenate([track_uris_tatums, [df["spotify_id"].iloc[k]]])

            for x, section in enumerate(data["sections"]): 
                
                songId_sections = np.concatenate([songId_sections, [k+1]])  
                numberSection_sections = np.concatenate([numberSection_sections, [x]])  
                start_sections = np.concatenate([start_sections, [section["start"]]])  
                duration_sections = np.concatenate([duration_sections, [section["duration"]]])  
                confidence_sections = np.concatenate([confidence_sections, [section["confidence"]]])  
                
                loudness_sections = np.concatenate([loudness_sections, [section["loudness"]]])  
                tempo_sections = np.concatenate([tempo_sections, [section["tempo"]]])  
                tempo_confidence_sections = np.concatenate([tempo_confidence_sections, [section["tempo_confidence"]]])  
                key_sections = np.concatenate([key_sections, [section["key"]]])  
                key_confidence_sections = np.concatenate([key_confidence_sections, [section["key_confidence"]]])  
                mode_sections = np.concatenate([mode_sections, [section["mode"]]])  
                mode_confidence_sections = np.concatenate([mode_confidence_sections, [section["mode_confidence"]]])  
                time_signature_sections = np.concatenate([time_signature_sections, [section["time_signature"]]]) 
                time_signature_confidence_sections = np.concatenate([time_signature_confidence_sections, [section["time_signature_confidence"]]])  

                track_uris_sections = np.concatenate([track_uris_sections, [df["spotify_id"].iloc[k]]])  

            for x, segments in enumerate(data["segments"]): 

                songId_segments = np.concatenate([songId_segments, [k+1]])  
                numberSegments_segments = np.concatenate([numberSegments_segments, [x]])  
                start_segments = np.concatenate([start_segments, [segments["start"]]]) 
                duration_segments = np.concatenate([duration_segments, [segments["duration"]]])  
                confidence_segments = np.concatenate([confidence_segments, [segments["confidence"]]])  
                loudness_start_segments = np.concatenate([loudness_start_segments, [segments["loudness_start"]]])  
                loudness_max_segments = np.concatenate([loudness_max_segments, [segments["loudness_max"]]])  
                loudness_max_time_segments = np.concatenate([loudness_max_time_segments, [segments["loudness_max_time"]]])  
                loudness_end_segments = np.concatenate([loudness_end_segments, [segments["loudness_end"]]])  
                
                pitches_segments.append(segments["pitches"])
                timbre_segments.append(segments["timbre"])  
                track_uris_segments = np.concatenate([track_uris_segments, [df["spotify_id"].iloc[k]]])  
            
            
            file_name = "Data/Features/High/" + str(k+1) + "-" + str(df["spotify_id"].iloc[k]).replace(" ","-").replace("/","-") + "-" + str(df["artist"].iloc[k]).replace(" ","-").replace("/","-") + ".json"
            
            with open(file_name, "w") as f:
                json.dump(data, f)

            k = k + 1   
        except:
            continue

In [ ]:
audio_features_high = {
    "analysis_time" : analysis_time, 
    "input_process" : input_process, 
    "num_samples" : num_samples, 
    "duration" : duration, 
    "sample_md5" : sample_md5, 
    "offset_seconds" : offset_seconds, 
    "window_seconds" : window_seconds,
    "analysis_sample_rate" : analysis_sample_rate, 
    "analysis_channels" : analysis_channels, 
    "end_of_fade_in" : end_of_fade_in, 
    "start_of_fade_out" : start_of_fade_out, 
    "loudness" : loudness, 
    "tempo" : tempo, 
    "tempo_confidence" : tempo_confidence, 
    "time_signature" : time_signature, 
    "time_signature_confidence" : time_signature_confidence, 
    "key" : key, 
    "key_confidence" : key_confidence, 
    "mode" : mode, 
    "mode_confidence" : mode_confidence, 
    "codestring" : codestring, 
    "code_version" : code_version, 
    "echoprintstring" : echoprintstring, 
    "echoprint_version" : echoprint_version, 
    "synchstring" : synchstring, 
    "synch_version" : synch_version, 
    "rhythmstring" : rhythmstring, 
    "rhythm_version" : rhythm_version, 
    "data_analysis" : data_analysis 
}
audio_features_high = pd.DataFrame(audio_features_high)

In [ ]:
bars = {
    'songId': songId_bars, 
    'numberBar': numberBar_bars, 
    'start': start_bars, 
    'duration': duration_bars, 
    'confidence': confidence_bars, 
    'track_uris': track_uris_bars
}
bars = pd.DataFrame(bars)

In [ ]:
beats = {
    'songId': songId_beats, 
    'numberBeat': numberBeat_beats, 
    'start': start_beats, 
    'duration': duration_beats, 
    'confidence': confidence_beats, 
    'track_uris': track_uris_beats
}
beats = pd.DataFrame(beats)

In [ ]:
tatums = {
    'songId': songId_tatums, 
    'numberTatum': numberTatum_tatums, 
    'start': start_tatums, 
    'duration': duration_tatums, 
    'confidence': confidence_tatums, 
    'track_uris': track_uris_tatums
}
tatums = pd.DataFrame(tatums)

In [ ]:
sections = {
    'songId': songId_sections, 
    'numberSection': numberSection_sections, 
    'start': start_sections, 
    'duration': duration_sections, 
    'confidence': confidence_sections, 
    'loudness': loudness_sections, 
    'tempo': tempo_sections, 
    'tempo_confidence': tempo_confidence_sections, 
    'key': key_sections, 
    'key_confidence': key_confidence_sections, 
    'mode': mode_sections, 
    'mode_confidence': mode_confidence_sections, 
    'time_signature': time_signature_sections, 
    'time_signature_confidence': time_signature_confidence_sections,  
    'track_uris': track_uris_sections
}
sections = pd.DataFrame(sections)

In [ ]:
segments = {
    "songId_segments":songId_segments,
    "numberSegments_segments":numberSegments_segments,
    "start_segments":start_segments,
    "duration_segments":duration_segments,
    "confidence_segments":confidence_segments,
    "loudness_start_segments":loudness_start_segments,
    "loudness_max_segments":loudness_max_segments,
    "loudness_max_time_segments":loudness_max_time_segments,
    "loudness_end_segments":loudness_end_segments,
    "pitches_segments":pitches_segments,
    "timbre_segments":timbre_segments,
    "track_uris_segments":track_uris_segments,
}
segments = pd.DataFrame(segments)

In [ ]:
audio_features_high.to_csv('Data/Data/data_caracteristicas_audio_high.csv', header=True, index=False)
bars.to_csv('Data/Data/data_bars.csv', header=True, index=False)
beats.to_csv('Data/Data/data_beats.csv', header=True, index=False)
tatums.to_csv('Data/Data/data_tatums.csv', header=True, index=False)
sections.to_csv('Data/Data/data_sections.csv', header=True, index=False)
segments.to_csv('Data/Data/data_segments.csv', header=True, index=False)